In [24]:
from IPython.display import HTML
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import requests
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import random
from fake_useragent import UserAgent

import csv
import openpyxl  # Library for creating XLSX files
import pandas as pd

# GET CHROME

In [29]:
def get_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    ua = UserAgent()
    user_agent = ua.random
    print(user_agent)
    # Adding argument to disable the AutomationControlled flag 
    chrome_options.add_argument("--disable-blink-features=AutomationControlled") 
    # Exclude the collection of enable-automation switches 
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
    # Turn-off userAutomationExtension 
    chrome_options.add_experimental_option("useAutomationExtension", False) 
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument(f'user-agent={user_agent}')
    # chrome_options.add_argument("--profile-directory=Profile 1")
    # user_dir = r'C:/Users/Admin/AppData/Local/Google/Chrome/User Data'
    # chrome_options.add_argument(f'--user-data-dir={user_dir}')
    driver = webdriver.Chrome(options=chrome_options, service=ChromeService(ChromeDriverManager().install()))
    return driver


# FILL PHONE NUMBER

In [52]:
driver = get_chrome_driver()
def process_excel_and_search(filename):
    try:
        # Read the Excel file with pandas, skipping the first row and selecting the first column
        data = pd.read_excel(filename, skiprows=[0], usecols=[0])
        # Extract data from the first column as a list
        search_terms = list(data.iloc[:, 0])
        data['Phone Number'] = ''
        print(search_terms)
        # Open Chrome and search for each term using webbrowser
        for i, term in enumerate(search_terms):
            phone_number = "Không có"
            search_url = f"https://www.google.com/search?q={term}"
            driver.get(search_url)
            time.sleep(1)
            phone_number = find_phone_number(driver)
            
            data.loc[i, 'Phone Number'] = phone_number
            print("Tên: "+ term, "Số điện thoại: " +phone_number)
            
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
    data.to_excel("output.xlsx", index=False)


def find_phone_number(chrome_driver):
    page_source = chrome_driver.page_source
    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    # # Find elements containing the text "số điện thoại" (case-insensitive)
    # spans_phone = soup.find_all('span', text=lambda text: text and "số điện thoại" in text.lower())

    # if spans_phone:
    #   for span in spans_phone:
    #         next_span = span.find_next_sibling('span')
    #         if(next_span):
    #             print("Next span:" + next_span.get_text())
    #             return next_span.get_text()
    try:
        element = chrome_driver.find_element(By.XPATH,"//*[contains(text(), 'Số điện thoại')]")
        if(element):
            #tìm div chính
            parent_div = element.find_element(By.XPATH, "./ancestor::div[1]")
            if(parent_div):
                spans = parent_div.find_elements(By.TAG_NAME, "span")
                if len(spans) >= 2:
                    if(spans[1].text != ":"):
                        return spans[1].text
                    else:
                        return spans[2].text
            
            # # Get the next sibling span element
            # next_span = element.find_element(By.XPATH, "following-sibling::span[1]")
            # if(next_span and next_span.text != ":"):
            #     return next_span.text
            # else:
            #     next_of_next_span = element.find_element(By.XPATH, "following-sibling::span[2]")
            #     if(next_of_next_span and next_of_next_span.text != ":"):
            #         return next_of_next_span.text
                
    except Exception as e:
        print(f"An error occurred when trying to find số điện thoại: {e}")
    return "Không có"
    


    

process_excel_and_search('data_quan_td.xlsx')

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15
['Lẩu Dê Thanh Bình - Đỗ Xuân Hợp', 'Lâm Nhựt - Cà Ri Dê Ấn Độ - Đường Số 12', 'Lẩu Dê Hiền Phước', 'Dê Núi Ninh Bình', 'Vĩnh Lộc - Quán Dê Tươi', 'Lẩu Dê Hòa Long 2', 'Dê Tươi Phong Phú - Shop Online', 'Đồng Xanh 6 - Đặc Sản Dê', 'Đức Tâm - Dê Núi Ninh Bình', 'Dê Tươi Ninh Bình - Man Thiện', 'Quán Dê Đông Triều - Dương Đình Hội', 'Dê Núi Ninh Thuận - Hoàng Hữu Nam', 'Lẩu Bò Đệ Tứ', 'Quán Mr Đèo - Bò Hít, Ram Bắp & Ram Chả Cá Lý Sơn']
Tên: Lẩu Dê Thanh Bình - Đỗ Xuân Hợp Số điện thoại: 0908 357 575
Tên: Lâm Nhựt - Cà Ri Dê Ấn Độ - Đường Số 12 Số điện thoại: 0918 069 069
Tên: Lẩu Dê Hiền Phước Số điện thoại: 0386 724 972
An error occurred when trying to find số điện thoại: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[contains(text(), 'Số điện thoại')]"}
  (Session info: chrome=126.0.6478.56); For documentation on this error, p

# AUTO CRAWL

In [54]:
def get_chrome_driver_with_profile():
    chrome_options = webdriver.ChromeOptions()
    ua = UserAgent()
    user_agent = ua.random
    print(user_agent)
    # Adding argument to disable the AutomationControlled flag 
    chrome_options.add_argument("--disable-blink-features=AutomationControlled") 
    # Exclude the collection of enable-automation switches 
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
    # Turn-off userAutomationExtension 
    chrome_options.add_experimental_option("useAutomationExtension", False) 
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--profile-directory=Profile 1")
    user_dir = r'C:/Users/Admin/AppData/Local/Google/Chrome/User Data'
    chrome_options.add_argument(f'--user-data-dir={user_dir}')
    driver = webdriver.Chrome(options=chrome_options, service=ChromeService(ChromeDriverManager().install()))
    return driver


driver = get_chrome_driver_with_profile()

def crawl_foody(url):
    driver.get(url)
    try:
        for _ in range(25):
            next_link = driver.find_element(By.CSS_SELECTOR, "a[href*='#page'][rel='next']")  # Using CSS selector
            if next_link:
                time.sleep(1)
                next_link.click()
                time.sleep(2)
            else:
                break
    except WebDriverException as e:
        print(f"WebDriverException: {e}")
    time.sleep(2)
    list_to_keep = ["bò", "lấu", "phá lấu", "lẩu bò", "dê", "dê nướng"]
    page_source = driver.page_source
    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    restaurant_data = []
    restaurant_elements = soup.find_all("div", class_="resname")
    print(f"Found {len(restaurant_elements)} restaurants")
    for restaurant in restaurant_elements:
        # Extract restaurant name (modify selector as needed)
        restaurant_name_element = restaurant.find("h2")  # Modify selector for name if needed
        address = restaurant.find('div', class_='address')
        spans = address.find_all('span')
        
        restaurant_name = restaurant_name_element.text.strip() if restaurant_name_element else ""  # Handle potential absence
        if not any(word.lower() in restaurant_name.lower() for word in list_to_keep):
            print("Skipping name: " + restaurant_name)
            continue
        dia_chi = ""
        quan = ""
        if(spans): 
            dia_chi = spans[0].get_text()
            quan = spans[1].get_text()
        restaurant_data.append([restaurant_name, dia_chi, quan])
        print("Tên:" + restaurant_name + "\tĐịa chỉ: "+ dia_chi + "\tQuận: " + quan)
    # Create XLSX workbook and worksheet
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.title = "Dữ liệu"
    # Write headers
    sheet.append(["Tên", "Địa Chỉ", "Quận"])
    # Write restaurant data
    for row in restaurant_data:
        sheet.append(row)
    # Save the XLSX file
    wb.save("data_quan_td.xlsx")


# Set the website URL
website_urlBNS = "https://www.foody.vn/ho-chi-minh/dia-diem?CategoryGroup=food&q=l%E1%BA%A9u%20b%C3%B2&dtids=1,4"
de_td_url = "https://www.foody.vn/ho-chi-minh/dia-diem-tai-tp-thu-duc?q=d%C3%AA&categorygroup=food"
# Số giây cách
refresh_interval = 3

# Gọi đến function auto refresh và auto click next chương
crawl_foody(de_td_url)

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15
Found 312 restaurants
Tên:Thành Đạt - Lẩu Dê Tài Ký	Địa chỉ: 5 Cây Keo P. Tam Phú	Quận: Tp. Thủ Đức
Tên:Lẩu Dê A Ty	Địa chỉ: 115 Liên Phường	Quận: Tp. Thủ Đức
Tên:Hệ thống Dê Thật	Địa chỉ: 

2 chi nhánh
                            	Quận: 
Tên:Nhà Hàng Tư Trì - Lẩu Dê & Hải Sản	Địa chỉ: 6 Khổng Tử, P. Bình Thọ	Quận: Tp. Thủ Đức
Tên:Lẩu Dê Đức Hiền - Hệ Thống Dê Núi Ninh Bình 7	Địa chỉ: 508 Lê Văn Việt, P. Tân Phú	Quận: Tp. Thủ Đức
Tên:Chảnh - Dê Tươi	Địa chỉ: 177 Trần Não	Quận: Tp. Thủ Đức
Tên:Lẩu Dê Phúc Dũng	Địa chỉ: 1453 Nguyễn Duy Trinh, P. Long Trường	Quận: Tp. Thủ Đức
Tên:HUGO QUÁN - Dê Quay Và Điểm Tâm Sáng	Địa chỉ: 34/1 Đinh Thị Thi, P. Hiệp Bình Phước 	Quận: Tp. Thủ Đức
Tên:Quang Thạnh - Lẩu Dê Tươi	Địa chỉ: Đường Liên Phường, P. Phước Long	Quận: Tp. Thủ Đức
Tên:Lang Liêu Quán - Chuyên Các Món Dê	Địa chỉ: 1363 Tỉnh Lộ 43, P. Bình Chiểu	Quận: Tp. Thủ Đức
Tên:Phú 